In [1]:
from LabelEncoder import LabelEncoder
from sklearn.preprocessing import LabelEncoder as skLE
import numpy as np
import pandas as pd

In [2]:
y_train = ["paris", "paris", "tokyo", '-1', -1.0, -999, 'foo']
y_test1 = ["paris", "tokyo", "paris", -999, np.nan]
y_test2 = ["paris", "paris", "tokyo", "tel aviv", "amsterdam", -999, np.nan]

### Default fit transform is equivalent to sklearn label encoder. 

In [3]:
skle = skLE()

In [4]:
skle.fit_transform(y_train)

array([4, 4, 5, 0, 1, 2, 3])

In [5]:
skle.classes_

array(['-1', '-1.0', '-999', 'foo', 'paris', 'tokyo'],
      dtype='|S5')

In [6]:
le = LabelEncoder()

In [7]:
le.fit_transform(y_train)

array([0, 0, 1, 3, 2, 5, 4])

In [ ]:
le.fit_transform(y_train).dtype

In [ ]:
le.classes

In [ ]:
le.most_common_class

In [ ]:
le.unknown_classes

### Fit transform with nan classes defined and retained. 

In [ ]:
le = LabelEncoder(retain_nan=True)

In [ ]:
y_train.append(np.nan)

In [ ]:
le.fit_transform(y_train)

### Fit and transform with different train and test data

In [ ]:
le = LabelEncoder(retain_nan=True)

In [ ]:
le.fit(y_train)

In [ ]:
le.classes

In [ ]:
le.transform(y_test1)

In [ ]:
le.transform(y_test2)

In [ ]:
le.unknown_classes

##

In [ ]:
le = LabelEncoder(retain_nan=False)

In [ ]:
le.fit(y_train)

In [ ]:
zz = le.classes
zz

In [ ]:
z = pd.Series(pd.Series(y_test1, dtype=np.object).unique())
z

In [ ]:
np.setdiff1d(z, zz)

In [ ]:
le.transform(y_test2)

In [ ]:
le.test_classes

In [ ]:
np.setdiff1d(le.test_classes, le.classes)